In [16]:
from typing_extensions import Annotated, TypedDict  

In [17]:
from langchain_openai import ChatOpenAI  

In [18]:
from typing import List

In [19]:
from tqdm import tqdm

In [20]:
import asyncio
from tqdm.asyncio import tqdm_asyncio

In [229]:
import pandas as pd

from langchain_core.vectorstores import InMemoryVectorStore  

from langchain_openai import OpenAIEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter  

from openai import OpenAI 

import requests

import json

from dotenv import load_dotenv
import os

# Подключаем все переменные из окружения
load_dotenv()
# Подключаем ключ для LLM-модели
LLM_API_KEY = os.getenv("LLM_API_KEY")
# Подключаем ключ для EMBEDDER-модели
EMBEDDER_API_KEY = os.getenv("EMBEDDER_API_KEY")

BASE_URL = 'https://openrouter.ai/api/v1'

client = OpenAI(
    base_url=BASE_URL,
    api_key=LLM_API_KEY,
)

question_df = pd.read_csv('questions.csv')

corpus = pd.read_csv('train_data.csv')

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=BASE_URL, api_key=LLM_API_KEY)

In [5]:
from pathlib import Path

In [121]:
chunks = []
folder = Path('./my_chunks_grok//')
for fn in os.listdir(folder):
    with open(folder / fn, 'r', encoding='utf-8') as f:
        chunks.append(f.read())

In [122]:
vectorstore = await InMemoryVectorStore.afrom_texts(    
    chunks,  
    embedding=embeddings, 
)

In [123]:
vectorstore.dump('context_chunking_grok.db')

In [124]:
class Query(TypedDict): 
    question: str
    problem: str
    answer_structure: str

In [125]:
class MyKAG:
    def __init__(self, model, store): 
        self.model = model
        self.store = store
        self.queries = []
        self.results = []

    def formulate_query(self, question):
        prompt = f"""
        Из заданного тебе вопроса сформулируй структурированный запрос на векторную базу данных.
        Отвечай кратко.
        question: вопрос без изменений
        problem: проблема, которая поставлена в вопросе
        answer_structure: структура ответа, которая полностью покроет заданный вопрос. Укажи темы, которые обязательно должны быть покрыты.
        Вопрос, на который нужно ответить:
        <question>
        {question}
        </question>"""
        query = self.model.with_structured_output(Query).invoke(prompt)
        self.queries.append({
            'question': question,
            'query': query
        })
        return query
    
    def get_knowledge(self, query):
        docs = self.store.similarity_search(query=str(query), k=20)
        knowledge = '\n'.join([doc.page_content for doc in docs])
        return knowledge

    def answer(self, question):
        query = self.formulate_query(question)
        knowledge = self.get_knowledge(query)
        prompt = f"""
        Ответьте на вопрос клиента, используя только релевантную предоставленную информацию. Не используйте не относящиеся к вопросу данные. Приведите ответ на русском языке.
        Если в вопросе есть запрос на какие-то численные данные, и эти данные есть в предоставленных вам знаниях, обязательно предоставьте их. Отвечайте на вопрос так, чтобы помочь клиенту.
        Ответ на вопрос ДОЛЖЕН БЫТЬ СВЯЗАН С проблемой поставленной в вопросе:
        <Problem>
        {query['problem']}
        </Problem>
        Используйте структуру ответа:
        <Structure>
        {query['answer_structure']}
        </Structure>
        <Question>
        {question}
        </Question>
        <Knowledge>
        {knowledge}
        </Knowledge>"""
        result = self.model.invoke(prompt).content
        self.results.append({
            'question': question,
            'query': query,
            'result': result,
            'knowledge': knowledge,
            'prompt': prompt
        })
        return result

In [126]:
model = ChatOpenAI(base_url=BASE_URL, model="x-ai/grok-3-mini", api_key=LLM_API_KEY)  

In [127]:
kag = MyKAG(model, vectorstore)

In [ ]:
results = []
for q in questions:
    kag.answer(q)

In [128]:
q = questions['Вопрос'].iloc[2]
q

'Как действовать при оспаривании незаконной продажи долга с учетом установленных сроков ответа на претензии?'

In [129]:
result = kag.answer(q) 

In [134]:
questions

,ID вопроса,Вопрос
0,1,Как просрочка по «беспроцентному» займу скажет...
1,2,"Как действовать вкладчику при отзыве лицензии,..."
2,3,Как действовать при оспаривании незаконной про...
3,4,Как изменился лимит социального вычета на спор...
4,5,Чем отличаются сроки обращения за наследством ...
...,...,...
495,496,В каких случаях банк обязан вернуть клиенту де...
496,497,В каких случаях банк вправе приостанавливать о...
497,498,Чем защита денег на банковских счетах отличает...
498,499,Что проверяют банки по бизнес-плану по сравнен...


In [254]:
class MyKAG:
    def __init__(self, model, store):
        self.model = model
        self.store = store
        self.queries = []
        self.results = []

    def formulate_query(self, question):
        prompt = f"""
        Из заданного вам вопроса сформулируйте структурированный запрос на векторную базу данных.
        Отвечайте кратко. В answer_structure также укажите то, что клиент хочет узнать из ответа на вопрос.
        question: вопрос без изменений
        problem: проблема, которая поставлена в вопросе
        answer_structure: структура ответа, которая полностью покроет заданный вопрос.
        Укажи темы, которые обязательно должны быть покрыты.
        Вопрос, на который нужно ответить:
        <question>
        {question}
        </question>
        """
        query = self.model.with_structured_output(Query).invoke(prompt)
        self.queries.append({'question': question, 'query': query})
        return query

    def get_knowledge(self, query):
        docs = self.store.similarity_search(query=str(query), k=20)
        knowledge = '\n'.join([doc.page_content for doc in docs])
        return knowledge

    def answer(self, question):
        print("[DEBUG] starting answer")
        print("[DEBUG] 1. formulating query")
        query = self.formulate_query(question)
        print("[DEBUG] 2. got query")
        
        print("[DEBUG] 3. retrieving knowledge")
        knowledge = self.get_knowledge(query)
        print("[DEBUG] 4. got knowledge")
    
        print("[DEBUG] 5. invoking model for answer")
        prompt = f"""
        Ответьте на вопрос клиента, используя только релевантную предоставленную информацию.
        Не используйте не относящиеся к вопросу данные. Приведите ответ на русском языке.
        Если в вопросе есть запрос на какие-то численные данные, и эти данные есть в предоставленных вам знаниях,
        обязательно предоставьте их. Отвечайте на вопрос так, чтобы помочь клиенту.
        Ответ на вопрос ДОЛЖЕН БЫТЬ СВЯЗАН С проблемой, поставленной в вопросе:
        <Problem>
        {query['problem']}
        </Problem>
        Используйте структуру ответа:
        <Structure>
        {query['answer_structure']}
        </Structure>
        <Question>
        {question}
        </Question>
        <Knowledge>
        {knowledge}
        </Knowledge>
        """
        print("[DEBUG] 6. got result")

        answer = self.model.invoke(prompt).content
        result = {
            'question': question,
            'query': query,
            'result': answer,
            'knowledge': knowledge,
            'prompt': prompt
        }
        self.results.append(result)
        return result

In [276]:
qs = list(question_df['Вопрос'])

In [277]:
qs[:10]

['Как просрочка по «беспроцентному» займу скажется на переплате/ПСК?',
 'Как действовать вкладчику при отзыве лицензии, учитывая лимит безопасной суммы?',
 'Как действовать при оспаривании незаконной продажи долга с учетом установленных сроков ответа на претензии?',
 'Как изменился лимит социального вычета на спорт по сравнению с 2022 годом?',
 'Чем отличаются сроки обращения за наследством по закону от сроков для налогового имущественного вычета?',
 'Почему при высокой долговой нагрузке банк может отказать в новом кредите?',
 'Что сделать первым делом при обнаружении мошеннического списания с карты?',
 'Куда обращаться, если заемные деньги ушли мошенникам?',
 'Как с помощью госгарантий повысить шанс одобрения и какую долю кредита они покрывают?',
 'Почему вклад безопаснее инвестиций в ценные бумаги?']

In [272]:
question_df['Вопрос'].nunique()

499

In [273]:
len(question_df['Вопрос'])

500

In [274]:
question_df['Вопрос'].value_counts()

Вопрос
Чем для вкладчика отличается санация от отзыва лицензии?                                                                2
Почему вклад безопаснее инвестиций в ценные бумаги?                                                                     1
Каковы сроки блокировки перевода, а также решения финомбудсмена?                                                        1
Почему иностранные депозитарные расписки и многие ETF могут быть заблокированы и кому они доступны?                     1
Какие приемы помогут сократить импульсные покупки?                                                                      1
                                                                                                                       ..
Как узнать, в каких БКИ хранится моя кредитная история, чтобы посмотреть свой индивидуальный рейтинг?                   1
Какие меры нужны для безопасных онлайн-платежей с детской карты?                                                        1
Какие действуют л

In [275]:
question_df[question_df['Вопрос'] == 'Чем для вкладчика отличается санация от отзыва лицензии?']

,ID вопроса,Вопрос
17,18,Чем для вкладчика отличается санация от отзыва...
133,134,Чем для вкладчика отличается санация от отзыва...


In [256]:
model = ChatOpenAI(base_url=BASE_URL, model="x-ai/grok-3-mini", api_key=LLM_API_KEY, temperature=0.1)
kag = MyKAG(model, vectorstore)

In [257]:
from concurrent.futures import ThreadPoolExecutor

In [292]:
async def answer_questions_with_executor(kag, questions, max_workers=20):
    loop = asyncio.get_running_loop()
    executor = ThreadPoolExecutor(max_workers=max_workers)

    tasks = []
    for i, q in enumerate(questions, start=1):
        # Directly schedule in executor; this returns a Future (awaitable)
        fut = loop.run_in_executor(executor, kag.answer, q)
        tasks.append(fut)
        print(f"[SCHED] Task {i}/{len(questions)} scheduled")

    results = []
    # Use as_completed to get progress updates
    for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Answering questions"):
        try:
            res = await f
            results.append(res)
            print("[OK] one result collected")
        except Exception as e:
            print("[ERROR] task raised:", repr(e))
            results.append(None)

    executor.shutdown(wait=True)
    return results


# Example runner
async def main(questions):
    model = ChatOpenAI(base_url=BASE_URL, model="x-ai/grok-3-mini", api_key=LLM_API_KEY)
    kag = MyKAG(model, vectorstore)
    print(">>> Starting concurrent run")
    results = await answer_questions_with_executor(kag, questions, max_workers=20)
    print(">>> Finished concurrent run, got", len(results), "results")
    return results

In [279]:
results = await main()

>>> Starting concurrent run
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 1/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 2/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 3/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 4/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 5/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 6/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 7/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 8/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 9/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 10/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 11/500 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 12

Answering questions:   0%|                                                                                                                                                                                 | 0/500 [00:00<?, ?it/s]

[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got resu

Answering questions:   0%|▎                                                                                                                                                                      | 1/500 [00:26<3:42:00, 26.69s/it]

[DEBUG] starting answer
[DEBUG] 1. formulating query
[OK] one result collected


...

[OK] one result collected


Answering questions: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 499/500 [13:36<00:02,  2.55s/it]

[OK] one result collected


Answering questions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [13:37<00:00,  1.64s/it]

[OK] one result collected
>>> Finished concurrent run, got 500 results


In [284]:
list(results[0].keys())

['question', 'query', 'result', 'knowledge', 'prompt']

In [286]:
kag

In [287]:
df = pd.DataFrame([x for x in results if x is not None])

In [288]:
len(df)

494

In [289]:
df.to_csv('submission_partial.csv')

In [294]:
unfinished_questions = list(set(df['question']) ^ set(question_df["Вопрос"]))

In [295]:
appendix = await main(unfinished_questions)

>>> Starting concurrent run
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 1/6 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 2/6 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 3/6 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 4/6 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 5/6 scheduled
[DEBUG] starting answer
[DEBUG] 1. formulating query
[SCHED] Task 6/6 scheduled


Answering questions:   0%|                                                                                                                                                                                   | 0/6 [00:00<?, ?it/s]

[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 2. got query
[DEBUG] 3. retrieving knowledge
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result
[DEBUG] 4. got knowledge
[DEBUG] 5. invoking model for answer
[DEBUG] 6. got result


Answering questions:  17%|████████████████████████████▌                                                                                                                                              | 1/6 [00:26<02:12, 26.40s/it]

[OK] one result collected


...

[OK] one result collected


Answering questions:  83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                            | 5/6 [00:36<00:04,  4.55s/it]

[OK] one result collected


Answering questions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:37<00:00,  6.29s/it]

[OK] one result collected
>>> Finished concurrent run, got 6 results


In [296]:
appendix_df = pd.DataFrame(appendix)

In [297]:
final_answer = pd.concat([appendix_df, df], axis=0)

In [298]:
final_answer.to_csv('public_submissin.csv') 

In [299]:
len(final_answer)

500

In [300]:
final_result = question_df.merge(final_answer, left_on='Вопрос', right_on='question', how='left')

In [302]:
final_result.drop_duplicates('result')

,ID вопроса,Вопрос,question,query,result,knowledge,prompt
0,1,Как просрочка по «беспроцентному» займу скажет...,Как просрочка по «беспроцентному» займу скажет...,{'question': 'Как просрочка по «беспроцентному...,### Влияние просрочки по беспроцентному займу ...,<title>\nИзменение ПСК в зависимости от обстоя...,"\n Ответьте на вопрос клиента, использу..."
1,2,"Как действовать вкладчику при отзыве лицензии,...","Как действовать вкладчику при отзыве лицензии,...",{'question': 'Как действовать вкладчику при от...,### Действия вкладчика при отзыве лицензии у б...,<title>\nУ меня счет в банке с отозванной лице...,"\n Ответьте на вопрос клиента, использу..."
2,3,Как действовать при оспаривании незаконной про...,Как действовать при оспаривании незаконной про...,{'question': 'Как действовать при оспаривании ...,### Оспаривание незаконной продажи долга: осно...,<title>\nКак быстро продадут жилье на торгах?\...,"\n Ответьте на вопрос клиента, использу..."
3,4,Как изменился лимит социального вычета на спор...,Как изменился лимит социального вычета на спор...,{'question': 'Как изменился лимит социального ...,### Изменения в лимите социального вычета на с...,<title>\nЗаниматься спортом станет выгоднее\n<...,"\n Ответьте на вопрос клиента, использу..."
4,5,Чем отличаются сроки обращения за наследством ...,Чем отличаются сроки обращения за наследством ...,{'question': 'Чем отличаются сроки обращения з...,### Сроки обращения за наследством по закону и...,<title>\nКакие долги передаются по наследству?...,"\n Ответьте на вопрос клиента, использу..."
...,...,...,...,...,...,...,...
497,496,В каких случаях банк обязан вернуть клиенту де...,В каких случаях банк обязан вернуть клиенту де...,{'question': 'В каких случаях банк обязан верн...,### В каких случаях банк обязан вернуть клиент...,<title>\nВозврат перевода по номеру карты или ...,"\n Ответьте на вопрос клиента, использу..."
498,497,В каких случаях банк вправе приостанавливать о...,В каких случаях банк вправе приостанавливать о...,{'question': 'В каких случаях банк вправе прио...,### В каких случаях банк вправе приостанавлива...,"<title>\nЧто делать, если банк отказывается пр...","\n Ответьте на вопрос клиента, использу..."
499,498,Чем защита денег на банковских счетах отличает...,Чем защита денег на банковских счетах отличает...,{'question': 'Чем защита денег на банковских с...,### Сравнение защиты денег на банковских счета...,<title>\nЧто дает НСЖ помимо страхования\n</ti...,"\n Ответьте на вопрос клиента, использу..."
500,499,Что проверяют банки по бизнес-плану по сравнен...,Что проверяют банки по бизнес-плану по сравнен...,{'question': 'Что проверяют банки по бизнес-пл...,### Сравнение критериев оценки бизнес-плана: б...,<title>\nВыбор способа финансирования проекта\...,"\n Ответьте на вопрос клиента, использу..."


In [304]:
very_final_result = final_result.drop_duplicates('result').drop(columns=['question', 'query', 'knowledge', 'prompt']).rename(columns={'result': 'Ответы на вопрос'})

In [306]:
very_final_result.to_csv('./submission/submission.csv', index=False)

In [309]:
final_result.to_csv('full_answers.csv')